In [17]:
import torch
import transformers
import onnx
import numpy as np
import onnxruntime.training.onnxblock as onnxblock
from onnxruntime.training import artifacts
from onnxruntime.training.onnxblock import blocks

In [18]:
feature_extractor = transformers.MobileViTFeatureExtractor.from_pretrained("apple/mobilevit-xx-small")
model = transformers.MobileViTForImageClassification.from_pretrained("apple/mobilevit-xx-small")

In [31]:
# model.classifier = torch.nn.Linear(320, 7)

onnx_name = "mobilevit_init_test.onnx"

# generates random pixel values for 5 images
# originally images were 256 by 256
# random_input = {"pixel_values": torch.rand(5, 3, 512, 512)}
random_input = torch.rand(5, 3, 256, 256)

torch.onnx.export(model, random_input, onnx_name,
                    input_names=["input"], output_names=["output"],
                    export_params=True,
                    dynamic_axes={
                        "input": {0: "batch_size"},
                        "output": {0: "batch_size"}
                    },
                    do_constant_folding=False,
                    training=torch.onnx.TrainingMode.TRAINING)

/bert_ort/carolinezhu/e2edemos/lib/python3.9/site-packages/torch/nn/functional.py:2415: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if size_prods == 1:
/bert_ort/carolinezhu/e2edemos/lib/python3.9/site-packages/transformers/models/mobilevit/modeling_mobilevit.py:452: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  new_height = int(math.ceil(orig_height / patch_height) * patch_height)
/bert_ort/carolinezhu/e2edemos/lib/python3.9/site-packages/transformers/models/mobilevit/modeling_mobilevit.py:453: TracerWarning: Converting a tensor to a Python float migh

In [21]:
requires_grad = []
frozen_params = []
for name, param in model.named_parameters():
    if param.requires_grad:
        requires_grad.append(name)
    else:
        frozen_params.append(name)

for name, param in model.named_buffers():
    frozen_params.append(name)

onnx_model = onnx.load(onnx_name)

artifacts.generate_artifacts(
    onnx_model,
    optimizer=artifacts.OptimType.AdamW,
    loss=artifacts.LossType.CrossEntropyLoss,
    requires_grad=requires_grad,
    frozen_params=frozen_params
)

RuntimeError: /bert_ort/carolinezhu/ort/onnxruntime/orttraining/orttraining/python/orttraining_pybind_state.cc:1068 onnxruntime::python::addObjectMethodsForTraining(pybind11::module&, onnxruntime::python::ExecutionProviderRegistrationFn)::<lambda(const pybind11::bytes&, const std::unordered_set<std::__cxx11::basic_string<char> >&)> [ONNXRuntimeError] : 1 : FAIL : Node (/mobilevit/encoder/layer.2/conv_projection/normalization/BatchNormalization_BatchNormInternal) Op (BatchNormInternal) [ShapeInferenceError] Dimension mismatch in unification between 48 and 1


In [23]:
from transformers import MobileViTFeatureExtractor, MobileViTForImageClassification
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

feature_extractor = MobileViTFeatureExtractor.from_pretrained("apple/mobilevit-xx-small")
model = MobileViTForImageClassification.from_pretrained("apple/mobilevit-xx-small")

inputs = feature_extractor(images=image, return_tensors="pt")

outputs = model(**inputs)
logits = outputs.logits
outputs.loss

In [30]:

inputs.pixel_values.shape

torch.Size([1, 3, 256, 256])

In [29]:
inputs.keys

<bound method BatchFeature.keys of {'pixel_values': tensor([[[[0.2471, 0.2745, 0.3294,  ..., 0.1529, 0.1490, 0.1451],
          [0.2510, 0.2471, 0.2745,  ..., 0.1529, 0.1373, 0.1412],
          [0.2392, 0.2510, 0.2863,  ..., 0.1333, 0.1333, 0.1451],
          ...,
          [0.7098, 0.7255, 0.7176,  ..., 0.7412, 0.7137, 0.7216],
          [0.7373, 0.7176, 0.7412,  ..., 0.7529, 0.7255, 0.7294],
          [0.7373, 0.7451, 0.7373,  ..., 0.7137, 0.6824, 0.7294]],

         [[0.1255, 0.1098, 0.1176,  ..., 0.0588, 0.0627, 0.0627],
          [0.1216, 0.1216, 0.1373,  ..., 0.0588, 0.0627, 0.0471],
          [0.1137, 0.1098, 0.1137,  ..., 0.0667, 0.0588, 0.0627],
          ...,
          [0.3216, 0.3098, 0.3373,  ..., 0.3373, 0.3569, 0.3255],
          [0.3216, 0.3294, 0.3255,  ..., 0.3451, 0.3451, 0.3294],
          [0.3216, 0.3569, 0.2980,  ..., 0.3490, 0.3529, 0.3176]],

         [[0.5922, 0.5804, 0.6039,  ..., 0.4078, 0.4078, 0.3922],
          [0.5804, 0.5686, 0.6039,  ..., 0.4078, 0.4118,

In [18]:
model.classifier = torch.nn.Linear(320, 7)